In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18936161
paper_name = 'mir_cashikar_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits_orfs_scores.txt', sep='\t', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 38 x 2


In [10]:
original_data.head()

,0,1
0,YLL026W,5
1,YCR009C,5
2,YLR414C,5
3,YPL101W,5
4,YDR293C,4


In [11]:
original_data['orf'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, orf]
Index: []


In [15]:
original_data['data'] = -original_data[1].astype(int)

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(38, 1)

# Load & process tested strains

In [22]:
tested = pd.read_excel('raw_data/Mat_a.xlsx', sheet_name='mat_a_041902', skiprows=1)

In [23]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,RG notes,QC Notes,Unnamed: 9
0,338.0,YAL068C,BY4741,01_1,1.0,A,2.0,NaN,NaN,NaN
1,339.0,YAL067C,BY4741,01_1,1.0,A,3.0,NaN,NaN,NaN
2,340.0,YAL066W,BY4741,01_1,1.0,A,4.0,NaN,NaN,NaN
3,341.0,YAL065C,BY4741,01_1,1.0,A,5.0,NaN,NaN,NaN
4,345.0,YAL062W,BY4741,01_1,1.0,A,6.0,NaN,NaN,NaN


In [24]:
tested['orf'] = tested['ORF name'].astype(str)

In [25]:
tested['orf'] = clean_orf(tested['orf'])

In [28]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [29]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [30]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name strain batch  plate  row  column RG notes  \
index_input                                                                  
4705                NaN      NaN    NaN   NaN    NaN  NaN     NaN      NaN   

            QC Notes Unnamed: 9  orf  
index_input                           
4705             NaN        NaN  NAN  


In [31]:
tested = tested.loc[t,:]

In [32]:
tested_orfs = tested['orf'].unique()

In [33]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [34]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [100]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,100
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,100
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [42]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [43]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        100       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18936161...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.79s/it]

Updating the data_modified_on field...
